In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

# 주제

    아파트 가격에 어떠한 요인이 영향을 미치는가?
    
      - 법정동코드에서 "서울특별시" 만 뽑고 읍면동에 대응하는 시군구명 뽑기.
      - 법정동코드에서 앞에 5자리(지역코드) 추출하기
      

# 1. 법정동 코드 가공

In [3]:
locCode = pd.read_excel('../data/법정동코드.xlsx')
locCode.head()

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,NaN,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,NaN,19880423,NaN
2,1111010100,서울특별시,종로구,청운동,NaN,19880423,NaN
3,1111010200,서울특별시,종로구,신교동,NaN,19880423,NaN
4,1111010300,서울특별시,종로구,궁정동,NaN,19880423,NaN


In [4]:
#읍면동이 NaN인 데이터 추출 > 읍면동 제외
locCode.dropna(subset=['읍면동명'],inplace=True)
locCode.head()

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자
2,1111010100,서울특별시,종로구,청운동,NaN,19880423,NaN
3,1111010200,서울특별시,종로구,신교동,NaN,19880423,NaN
4,1111010300,서울특별시,종로구,궁정동,NaN,19880423,NaN
5,1111010400,서울특별시,종로구,효자동,NaN,19880423,NaN
6,1111010500,서울특별시,종로구,창성동,NaN,19880423,NaN


In [62]:
# 시군구가 NaN이 아닌 데이터 추출 > 시도명 제외

locCode.dropna(subset=['시군구명'],inplace=True)
locCode.head()

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자
2,1111010100,서울특별시,종로구,청운동,NaN,19880423,NaN
3,1111010200,서울특별시,종로구,신교동,NaN,19880423,NaN
4,1111010300,서울특별시,종로구,궁정동,NaN,19880423,NaN
5,1111010400,서울특별시,종로구,효자동,NaN,19880423,NaN
6,1111010500,서울특별시,종로구,창성동,NaN,19880423,NaN


In [5]:
# 법정동 코드 컬럼에서 5자리 추출합니다.
# 예) 서울특별시 종로구 > 11000
locCode['법정동코드']=locCode['법정동코드'].astype(str)
locCode['지역코드']=locCode['법정동코드'].apply(lambda x: x[:5])
locCode.head()




,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자,지역코드
2,1111010100,서울특별시,종로구,청운동,NaN,19880423,NaN,11110
3,1111010200,서울특별시,종로구,신교동,NaN,19880423,NaN,11110
4,1111010300,서울특별시,종로구,궁정동,NaN,19880423,NaN,11110
5,1111010400,서울특별시,종로구,효자동,NaN,19880423,NaN,11110
6,1111010500,서울특별시,종로구,창성동,NaN,19880423,NaN,11110


In [12]:
# 시도명이 서울특별시인 데이터만 추출 후 시도명, 시군구명, 지역코드 컬럼만 추출 

locCode=locCode[locCode['시도명']=='서울특별시']

loc=pd.DataFrame({'시도명':locCode['시도명'],'시군구명':locCode['시군구명'], '지역코드':locCode['지역코드'],'법정동명':locCode['읍면동명']})
loc


,시도명,시군구명,지역코드,법정동명
2,서울특별시,종로구,11110,청운동
3,서울특별시,종로구,11110,신교동
4,서울특별시,종로구,11110,궁정동
5,서울특별시,종로구,11110,효자동
6,서울특별시,종로구,11110,창성동
...,...,...,...,...
488,서울특별시,강동구,11740,둔촌동
489,서울특별시,강동구,11740,암사동
490,서울특별시,강동구,11740,성내동
491,서울특별시,강동구,11740,천호동


# 2. 실거래가 데이터 가공

In [7]:
# 실거래가 데이터 불러오기
apt = pd.read_csv('../data/2020년 서울 아파트 실거래가.csv', encoding="cp949")
apt.head()

,법정동명,건물면적,층정보,건물주용도코드,건물주용도,거래금액,건축년도,아파트명
0,장위동,59.92,5.0,2001,아파트,749000000,0.0,래미안 장위포레카운티
1,길음동,84.77,18.0,2001,아파트,1200000000,0.0,롯데캐슬 클라시아
2,구로동,36.90,1.0,2001,아파트,105000000,1994.0,궁전아트빌라
3,산천동,59.55,2.0,2001,아파트,830000000,2001.0,리버힐삼성
4,이촌동,64.43,5.0,2001,아파트,735000000,1971.0,강변


In [106]:
# 데이터 프레임의 컬럼을 확인합니다.
apt.columns

Index(['법정동명', '건물면적', '층정보', '건물주용도코드', '건물주용도', '거래금액', '건축년도', '아파트명'], dtype='object')

In [107]:
# 데이터 프레임의 크기(차원)을 조회합니다
apt.shape

(176001, 8)

In [108]:
# 불러온 데이터에 NA값이 있는지 조회합니다.
apt.isnull().sum(axis=1)

0         0
1         0
2         0
3         0
4         0
         ..
175996    2
175997    2
175998    0
175999    2
176000    2
Length: 176001, dtype: int64

In [109]:
# 실거래가 데이터프레임 정보 조회
apt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176001 entries, 0 to 176000
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   법정동명     176001 non-null  object 
 1   건물면적     176001 non-null  float64
 2   층정보      163838 non-null  float64
 3   건물주용도코드  176001 non-null  int64  
 4   건물주용도    176001 non-null  object 
 5   거래금액     176001 non-null  int64  
 6   건축년도     175436 non-null  float64
 7   아파트명     163838 non-null  object 
dtypes: float64(3), int64(2), object(3)
memory usage: 10.7+ MB


In [110]:
# 실거래가 데이터프레임 정보 조회 (거래금액 컬럼 변경 확인)
apt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176001 entries, 0 to 176000
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   법정동명     176001 non-null  object 
 1   건물면적     176001 non-null  float64
 2   층정보      163838 non-null  float64
 3   건물주용도코드  176001 non-null  int64  
 4   건물주용도    176001 non-null  object 
 5   거래금액     176001 non-null  int64  
 6   건축년도     175436 non-null  float64
 7   아파트명     163838 non-null  object 
dtypes: float64(3), int64(2), object(3)
memory usage: 10.7+ MB


In [8]:
# 범주화된 전용면적의 고유값을 확인합니다.

apt['건물면적'] = apt['건물면적'].astype(int) // 3.3057
apt = apt.rename({'건물면적':'평수'},axis='columns')
apt

,법정동명,평수,층정보,건물주용도코드,건물주용도,거래금액,건축년도,아파트명
0,장위동,17,5.0,2001,아파트,749000000,0.0,래미안 장위포레카운티
1,길음동,25,18.0,2001,아파트,1200000000,0.0,롯데캐슬 클라시아
2,구로동,10,1.0,2001,아파트,105000000,1994.0,궁전아트빌라
3,산천동,17,2.0,2001,아파트,830000000,2001.0,리버힐삼성
4,이촌동,19,5.0,2001,아파트,735000000,1971.0,강변
...,...,...,...,...,...,...,...,...
175996,상암동,88,NaN,1001,단독주택,1450000000,1998.0,NaN
175997,연남동,44,NaN,1001,단독주택,2430000000,1973.0,NaN
175998,망원동,13,3.0,2002,연립주택,405000000,1983.0,(459-4)
175999,신수동,13,NaN,1001,단독주택,899900000,1967.0,NaN


In [45]:
# apt=apt[apt['건물주용도']=='아파트']
apt.isnull().sum()



법정동명     0
평수       0
층정보      0
건물주용도    0
거래금액     0
건축년도     0
아파트명     0
dtype: int64

In [37]:
apt.drop(columns=['건물주용도코드'],inplace=True)
apt

KeyError: "['건물주용도코드'] not found in axis"

# 3. 데이터 프레임 결합

In [47]:

# 지역 코드를 기준으로 법정동 코드 데이터 프레임과 아파트 실거래가 데이터 프레임을 결합합니다.
df = pd.merge(loc,apt)
df.drop_duplicates(inplace=True)


df.pivot_table('거래금액',index=['시군구명'],aggfunc='mean').sort_values('거래금액')

,거래금액
시군구명,
도봉구,4.566446e+08
금천구,4.943751e+08
중랑구,5.075473e+08
노원구,5.169475e+08
구로구,5.424521e+08
강북구,5.549278e+08
관악구,6.252989e+08
은평구,6.426621e+08
강서구,6.732522e+08
